In [2]:
import os
import torch 
curr = os.path.basename(os.getcwd())
print(curr)
os.chdir("../../")
print(os.getcwd())

UNet_CE
/home/oem/Documents/GoogleStreetSegmentation/www


In [2]:
map_location=torch.device('cpu')

model = torch.load(f"models/{curr}/best_model", map_location=map_location)
model.cpu()
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))



31033435


/home/oem/.local/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/oem/.local/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/oem/.local/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.pooling.MaxPool2d' has changed. you can retrieve the original source code by accessing the object's source at

# Convert to onnx

In [3]:
dummy_input = torch.zeros([1,3,512,512])
torch.onnx.export(model, dummy_input, f"models/{curr}/model.onnx", 
                  verbose=True)

graph(%input.1 : Float(1, 3, 512, 512, strides=[786432, 262144, 512, 1], requires_grad=0, device=cpu),
      %conv1_1.weight : Float(64, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %conv1_1.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %conv1_2.weight : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %conv1_2.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %conv2_1.weight : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %conv2_1.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %conv2_2.weight : Float(128, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cpu),
      %conv2_2.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %conv3_1.weight : Float(256, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cpu),
      %conv3_1.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %conv3_2.weight : 

# Convert to TF

In [3]:
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf



In [6]:
onnx_model = onnx.load(f"models/{curr}/model.onnx")
tf_rep = prepare(onnx_model)
tf_rep.export_graph(f"output/model{curr}.pb")

INFO:tensorflow:Assets written to: output/modelUNet_CE.pb/assets


INFO:tensorflow:Assets written to: output/modelUNet_CE.pb/assets


In [7]:
cmd = f"tensorflowjs_converter --input_format=tf_saved_model  output/model{curr}.pb output/model{curr}.pb"
os.system(cmd)

2021-12-05 00:00:33.636230: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-05 00:00:33.636255: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-05 00:00:34.692259: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 00:00:34.692902: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-05 00:00:34.692945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

Writing weight file output/modelUNet_CE.pb/model.json...


0